# DIET on the DAGHAR Dataset

In this notebook, we will pre-train a CNN_PF encoder using Datum IndEx as Target (DIET) on the DAGHAR KuHAR dataset, then fine-tune it on the same dataset. Finally, we will evaluate the model's performance using a test set. Note that while this example performs both pre-training and fine-tuning on the same dataset, in practical scenarios, pre-training typically involves a larger, more diverse dataset.

The following core libraries from `minerva` will be used:

1. **Pre-Training**:
   - [`minerva.models.ssl.cpc.CPC`](): Implements CPC as a PyTorch Lightning module.
   - [`minerva.models.nets.tnc.TSEncoder`](): Implements the TS2Vec encoder (our "backbone"), which will be trained using CPC (`g_enc`).
   - [`minerva.models.nets.cpc_networks.HARCPCAutoregressive`](): Implements the CPC autoregressive network (`g_ar`), the default autoregressive network used in CPC for human activity recognition.
   - [`minerva.data.data_modules.har_rodrigues_24.HARDataModuleCPC`](): A Lightning DataModule that loads and organizes the DAGHAR KuHAR dataset for CPC training, providing training, validation, and test data loaders.
   - [`minerva.pipelines.lightning_pipeline.SimpleLightningPipeline`](): A wrapper for PyTorch Lightning’s `fit` method, enhancing reproducibility, logging, and customization for analytical purposes.
2. **Fine-Tuning**:
   - [`minerva.models.nets.base.SimpleSupervisedModel`](): Implements a supervised model with a customizable backbone (the pre-trained TS2Vec encoder) and head. This setup allows backbone freezing and supports a flexible head structure.
      - `FromPretrained`: Loads only the backbone (TS2Vec) from the checkpoint saved during pre-training.
      - [`minerva.models.nets.mlp.MLP`](): Defines a simple MLP classifier used as the head of the supervised model.
   - [`minerva.pipelines.lightning_pipeline.SimpleLightningPipeline`](): The same pipeline wrapper used in pre-training, with additional support for custom evaluation metrics and analysis.
   - [`minerva.data.data_modules.har.MultiModalHARSeriesDataModule`](): A Lightning DataModule for loading data in the format required for fine-tuning, organized similarly to the DAGHAR dataset. This module provides sliding window time series data suitable for fine-tuning.

**Note**: 
1. Although we use the DAGHAR dataset here, this pipeline is flexible and can be adapted for other datasets. 
2. The data module configuration differs between pre-training (using full time series for each sample) and fine-tuning (using sliding windows) to suit model requirements at each stage.

### Useful Links:
- [DAGHAR Dataset on Zenodo](https://zenodo.org/records/13987073)
- [Contrastive Predictive Coding for Human Activity Recognition]()
- [TS2Vec: Towards Universal Representation of Time Series](https://cdn.aaai.org/ojs/20881/20881-13-24894-1-2-20220628.pdf)

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from datetime import datetime

import lightning as L
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from minerva.models.ssl.cpc import CPC
from minerva.models.nets.cpc_networks import HARCPCAutoregressive
from minerva.data.data_modules.har_rodrigues_24 import HARDataModuleCPC

from minerva.models.ssl.diet import DIET
from minerva.models.nets.diet_linear import DIETLinear,AdaptedHead

from minerva.pipelines.lightning_pipeline import SimpleLightningPipeline
from minerva.models.nets.base import SimpleSupervisedModel

from minerva.models.nets.tnc import TSEncoder
import torchmetrics

from minerva.data.data_modules.har import MultiModalHARSeriesDataModule
from minerva.models.loaders import FromPretrained
from minerva.models.nets.base import SimpleSupervisedModel
from minerva.models.nets.mlp import MLP
from minerva.analysis.metrics.balanced_accuracy import BalancedAccuracy
from minerva.analysis.model_analysis import TSNEAnalysis
from minerva.models.nets.tnc import RnnEncoder

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
execution_id = f'run_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
log_dir = f"./logs/{execution_id}" 

print(f"Execution ID: {execution_id}")
print(f"Log dir: {log_dir}")

Execution ID: run_20260127-022917
Log dir: ./logs/run_20260127-022917


## Pre-training with DIET

We will train a CNN_PF Encoder using [DAGHAR's KuHAr dataset](https://zenodo.org/records/13987073) and Datum IndEx as Target (DIET) as the pretext task. 

### 1. Defining the Data Module

We will use the `HARDataModuleCPC` data module to load the DAGHAR dataset for the backbone pretraining. This data module loads the data in the format required for training, which includes full time series data for each sample.

Required arguments:
- `data_path`: Path to the directory containing the dataset.
- `input_size`: The number of features in the input time series.
- `window_size`: The size of the sliding window used to create the positive and negative samples for CPC.
- `overlap`: The overlap between consecutive windows.
- `batch_size`: The batch size for training.

In [4]:
data_module = HARDataModuleCPC(
    data_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/rodrigues_2024_datasets/1-1/kuhar",
    input_size=6,
    window=60,
    overlap=60,
    batch_size=64,
    label="return_index_as_label",
    use_val_with_train = True
)

data_module

HARDataModuleCPC(batch_size=64, datasets=/workspaces/HIAAC-KR-Dev-Container/shared_data/rodrigues_2024_datasets/1-1/kuhar)

In [5]:
train_size = len(data_module.train_dataloader().dataset)
print(f"Train size: {train_size}")

Train size: 1824


### 2. Defining the DIET Model

We will create the CPC model using the TS2Vec encoder as the backbone and the HARCPCAutoregressive network as the autoregressive network.

The TS2Vec encoder (`g_enc`) requires the following arguments:
- `input_dims`: The number of features in the input time series.
- `output_dims`: The dimensionality of the output embeddings.
- `depth`: Number of convolutional layers.
- `permute`: Whether to permute the input time series before passing it through the encoder.

The HARCPCAutoregressive network (`g_ar`) requires the following arguments:
- `input_size`: The dimensionality of the input embeddings.
- `hidden_size`: The dimensionality of the hidden state in the autoregressive network.
- `batch_first`: Whether the input is batch-first.
- `bidirectional`: Whether the autoregressive network is bidirectional.

The `CPC` model requires the following arguments:
- `g_enc`: The TS2Vec encoder.
- `g_ar`: The CPC autoregressive network.
- `prediction_head_in_channels`: The dimensionality of the input to the prediction head.
- `prediction_head_out_channels`: The dimensionality of the output of the prediction head.
- `num_steps_prediction`: The number of steps to predict in the future.
- `batch_size`: The batch size for training.
- `minimum_steps`: The minimum number of steps in the future to predict.

In [7]:
from minerva.models.nets.time_series.cnns import CNN_PF_Backbone

backbone = CNN_PF_Backbone(include_middle=True,flatten=True)
projection_head = DIETLinear(in_features = 768,out_features = train_size)

model = DIET(
    backbone=backbone,
    # encoding_size=768,
    # num_data=train_size,



    # linear_layer=projection_head,
    # scheduler="WarmupCosineAnnealingLR",
    # wca_scheduler_total_epochs=None,
    learning_rate=3e-4,
    weight_decay=3e-4,
)
model

DIET(
  (backbone): CNN_PF_Backbone(
    (first_padder): ZeroPadder2D(pad_at=(3,), padding_size=2)
    (upper_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (lower_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (middle_part): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
    (shared_part): Sequential(
      (0): Conv2d(48, 64, kernel_size=(3, 5), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=(2, 3), stride=(2, 3), padding=1, dilation=1, ceil_mode=False)
    )
  )
  (loss): CrossEntropyLoss()
)

### 3. Defining the Pytorch Lightning Trainer Configuration

We will define the PyTorch Lightning Trainer configuration for training the CPC model. This configuration includes the following parameters:
- `max_epochs`: The maximum number of epochs for training.
- `accelartor`: The device to use for training (e.g., 'cuda' or 'cpu').
- `devices`: The number (or a list) of accelerator devices to use.
- `logger`: The logger to use for logging training metrics.
- `callbacks`: The callbacks to use during training. Callbacks allows customizing the training loop and adding additional functionality, such as early stopping, model checkpointing, and learning rate scheduling.
- `limit_*_batches`: The number of batches to limit the training, validation, and testing datasets. This is useful for debugging and testing the pipeline.

The logger and callbacks can be customized based on the requirements of the training pipeline.
We will use the `CSVLogger` callback to log the training metrics to a CSV file and the `ModelCheckpoint` callback to save the best model based on the validation loss.

In [8]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    # monitor='train_loss',
    # mode='min',
    # save_last=True
    filename='best'
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='diet-pretraining', version=execution_id)

## Trainer
trainer = L.Trainer(
    max_epochs=5,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    # limit_train_batches=1,
    # limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


### 4. Creating the training pipeline (and running the training

We will create a `SimpleLightningPipeline` to train the CPC model. This pipeline provides a high-level interface for training the model, logging the training metrics, and saving the best model based on the validation loss. Also, it allows customizing the evaluation metrics and analysis based on the requirements.

The pipeline requires the following arguments:
- `model`: The CPC model to train.
- `trainer`: The PyTorch Lightning Trainer configuration.
- `log_dir`: The directory to save the training logs and model checkpoints.
- `seed`: The random seed for reproducibility.
- `save_run_stats`: Whether to save the training statistics for reproducibility and analysis.

Once the pipeline is created, we can call the `run` method with the data module to start the training process. The parameter `task` of `run` method should be set to:
- `fit`: To train the model.
- `evaluate`: To evaluate the model on the test set.

In [9]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


** Seed set to: 42 **
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/codecarbon/logs/run_20260127-022917/run_2026-01-27-02-30-047fe44bdd.yaml



  | Name        | Type             | Params | Mode 
---------------------------------------------------------
0 | backbone    | CNN_PF_Backbone  | 46.6 K | train
1 | loss        | CrossEntropyLoss | 0      | train
2 | linear_head | Linear           | 1.4 M  | train
---------------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.797     Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (28) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 4: 100%|██████████| 28/28 [00:00<00:00, 28.40it/s, v_num=2917]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 28/28 [00:01<00:00, 21.03it/s, v_num=2917]
⏱️ fit took 7.04s → saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/codecarbon/logs/run_20260127-022917/timings_fit.csv
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/codecarbon/logs/run_20260127-022917/run_2026-01-27-02-30-047fe44bdd.yaml


### 5. Inspecting Checkpoints

Once the training is complete, we can inspect the saved checkpoints to load the pre-trained model for fine-tuning or further analysis.

Let's inspect the checkpoint file from the trained CPC model and check the weights that correspond to the TS2Vec encoder (`g_enc`).

If you check the output, the checkpoint keys that starts with `g_enc` are the weights of the TS2Vec encoder. The `g_ar` weights are also saved in the checkpoint, and also the `prediction_head` weights. It is important to note that the `g_ar` and `prediction_head` weights are not used in the fine-tuning process, as we will only load the TS2Vec encoder for fine-tuning. Thus, we will only load the weights corresponding to the `g_enc` keys.

In [10]:
ckpt_path = checkpoint_callback.best_model_path
ckpt = torch.load(ckpt_path, map_location="cuda")
ckpt = ckpt.get("state_dict", ckpt)
list(ckpt.keys())

['backbone.upper_part.0.weight',
 'backbone.upper_part.0.bias',
 'backbone.lower_part.0.weight',
 'backbone.lower_part.0.bias',
 'backbone.middle_part.0.weight',
 'backbone.middle_part.0.bias',
 'backbone.shared_part.0.weight',
 'backbone.shared_part.0.bias',
 'linear_head.weight',
 'linear_head.bias']

## Fine-tuning the Pre-trained TS2Vec Encoder

After pre-training the TS2Vec encoder using CPC, we will fine-tune the encoder on the DAGHAR dataset for human activity recognition. We will use the same DAGHAR dataset but with a different data module configuration that provides sliding window time series data suitable for fine-tuning.

### 1. Defining the Data Module

We will use the `MultiModalHARSeriesDataModule` data module to load the DAGHAR dataset for fine-tuning. This data module loads the data in the format required for fine-tuning, which includes sliding window time series data for each sample. Thus, each sample of the dataset will be a 2-element tuple containing the time series (6x60, where 6 is the number of features and 60 is the window size) and the corresponding label.

The data module requires the following arguments:
- `data_path`: Path to the directory containing the dataset.
- `feature_prefix`: The prefix of the columns containing the features. For each prefix, we will create a different channel with all columns that start with the prefix.
- `label`: The name of the column containing the labels.
- `features_as_channels`: If True, for each prefix, we will create a different channel with all columns that start with the prefix. If False, we will concatenate all columns with the same prefix into a single channel (the sample will be a tensor of 1x360 instead of 6x60).
- `cast_to`: The data type to cast the features (float32).
- `batch_size`: The batch size for training.

In [11]:
data_module = MultiModalHARSeriesDataModule(
    data_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/KuHar/",
    feature_prefixes=["accel-x", "accel-y", "accel-z", "gyro-x", "gyro-y", "gyro-z"],
    label="standard activity code",
    features_as_channels=True,
    cast_to="float32",
    batch_size=64,
)

data_module

MultiModalHARSeriesDataModule(data_path=/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/KuHar, batch_size=64)

In [12]:
# Pega os dataloaders de treino e validação
data_module.setup("fit")
train_data_loader = data_module.train_dataloader()
validation_data_loader = data_module.val_dataloader()
first_batch = next(iter(train_data_loader))

X, y = first_batch
print(X.shape, y.shape)

Using DataLoader with shuffle=True
Using DataLoader with shuffle=False
torch.Size([64, 6, 60]) torch.Size([64])


In [13]:
from  minerva.models.nets.cpc_networks import Genc_Gar

In [14]:
model_ = CNN_PF_Backbone(include_middle=True,flatten=True)

### 2. Defining the Fine-tuning Model

We first load the pre-trained TS2Vec encoder from the checkpoint saved during pre-training. We then create the supervised model with the TS2Vec encoder as the backbone and an MLP classifier as the head.

To load the TS2Vec encoder from the checkpoint, we use the `FromPretrained` class, which loads only the backbone from the checkpoint. The `FromPretrained` class requires the following arguments:
- `model`: The encoder model, with randomly initialized weights.
- `ckpt_path`: The path to the checkpoint file.
- `filter_keys`: A list of keys to filter from checkpoint keys. We will use this argument to load only the weights corresponding to the TS2Vec encoder (The keys that start with `g_enc`).

In [15]:
# g_enc = TSEncoder(input_dims=6, output_dims=64, hidden_dims=64, depth=10, permute=True)
backbone = FromPretrained(
    model=model_,#g_enc,
    ckpt_path=checkpoint_callback.best_model_path,
    filter_keys=["backbone"],
    keys_to_rename={"backbone.": ""}, 
    # keys_to_rename= { "g_enc.":"","g_ar.":""},
    strict=True,
    error_on_missing_keys=True
)

Performing key renaming with: {'backbone.': ''}
	Renaming key: backbone.upper_part.0.weight -> upper_part.0.weight (changed: True)
	Renaming key: backbone.upper_part.0.bias -> upper_part.0.bias (changed: True)
	Renaming key: backbone.lower_part.0.weight -> lower_part.0.weight (changed: True)
	Renaming key: backbone.lower_part.0.bias -> lower_part.0.bias (changed: True)
	Renaming key: backbone.middle_part.0.weight -> middle_part.0.weight (changed: True)
	Renaming key: backbone.middle_part.0.bias -> middle_part.0.bias (changed: True)
	Renaming key: backbone.shared_part.0.weight -> shared_part.0.weight (changed: True)
	Renaming key: backbone.shared_part.0.bias -> shared_part.0.bias (changed: True)
Model loaded from /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/codecarbon/checkpoints/best.ckpt


Once backbone is loaded, we create the supervised model with the TS2Vec encoder as the backbone and an MLP classifier as the head. The MLP classifier requires the following arguments:
- A list with layer sizes for the MLP. The first element should be the size of the input layer (the output of the TS2Vec encoder), and the last element should be the size of the output layer (the number of classes). We use a MLP with 3840 input units (the output of the TS2Vec encoder), a hidden layer with 128 units, and an output layer with 6 units (the number of classes in the DAGHAR dataset).

In [16]:
head = MLP([768, 128, 6])

Finally, we use the `SimpleSupervisedModel` class to create the supervised model with the TS2Vec encoder as the backbone and the MLP classifier as the head. The `SimpleSupervisedModel` class requires the following arguments:
- `backbone`: The backbone model (the pre-trained TS2Vec encoder).
- `fc`: The head model (the MLP classifier).
- `loss_fn`: The loss function to use for training (CrossEntropyLoss).
- `flatten`: Whether to flatten the input before passing it through the head. Usually, the input is flattened if the backbone outputs a tensor with more than two dimensions.
- `train_metrics`: A dictionary where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.
- `val_metrics`: A dictionary where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.

In [17]:
# model = SimpleSupervisedModel(
#     backbone=backbone,
#     fc=head,
#     loss_fn=torch.nn.CrossEntropyLoss(),
#     flatten=False,
#     train_metrics={
#         "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
#     },
#     val_metrics={
#         "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
#     },
# )

# model


# from minerva.models.adapters import MaxPoolingTransposingSqueezingAdapter

# adapter = MaxPoolingTransposingSqueezingAdapter(kernel_size=64)
model = SimpleSupervisedModel(
    backbone=backbone,
    fc=head,
    loss_fn=torch.nn.CrossEntropyLoss(),
    flatten=False,
    freeze_backbone=True,
    # adapter = adapter,
    train_metrics={
        "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
    },
    val_metrics={
        "acc": torchmetrics.Accuracy(task="multiclass", num_classes=6),
    },
)

In [18]:
# Option 1: Measure ALL model parameters (including backbone AND head)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total model parameters: {total_params:,}")
print(f"Trainable model parameters: {trainable_params:,}")

# Option 2: Measure components separately (as you intended)
# Backbone parameters
backbone_total = sum(p.numel() for p in model.backbone.parameters())
backbone_trainable = sum(p.numel() for p in model.backbone.parameters() if p.requires_grad)

# Head parameters (use model.fc, not head variable)
head_total = sum(p.numel() for p in model.fc.parameters())
head_trainable = sum(p.numel() for p in model.fc.parameters() if p.requires_grad)

print(f"\nBackbone:")
print(f"  Total parameters: {backbone_total:,}")
print(f"  Trainable parameters: {backbone_trainable:,}")

print(f"\nClassification Head:")
print(f"  Total parameters: {head_total:,}")
print(f"  Trainable parameters: {head_trainable:,}")

# Verify the sum matches Option 1
print(f"\nVerification:")
print(f"Backbone + Head total: {backbone_total + head_total:,}")
print(f"Should match total: {total_params:,}")

from thop import profile
evaluation_data = torch.rand(1000, 6, 60, device='cuda')
model.to('cuda')
macs, params = profile(model, inputs=(evaluation_data,))

print(f"MACs: {macs:,}")
print(f"Parameters: {params:,}")

# from codecarbon import EmissionsTracker

torch.cuda.is_available()
torch.cuda.device_count()




Total model parameters: 145,830
Trainable model parameters: 145,830

Backbone:
  Total parameters: 46,624
  Trainable parameters: 46,624

Classification Head:
  Total parameters: 99,206
  Trainable parameters: 99,206

Verification:
Backbone + Head total: 145,830
Should match total: 145,830
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
MACs: 1,673,856,000.0
Parameters: 145,830.0


1

In [19]:
import pandas as pd
from codecarbon import EmissionsTracker

TOTAL_RUNS = 14
DISCARD_FIRST = 2
DISCARD_LAST = 2

results = []

tracker = EmissionsTracker(
    project_name="basic_measurement",
    measure_power_secs=10,
    save_to_file=False
)

try:
    for run_id in range(TOTAL_RUNS):
        tracker.start_task(f"measure_inference_{run_id}")

        _ = model(evaluation_data)  # inference

        emissions = tracker.stop_task()

        energy_kwh = emissions.energy_consumed
        energy_mwh = energy_kwh * 1_000          # kWh → mWh
        energy_j   = energy_kwh * 3_600_000      # kWh → J
        emissions_g = emissions.emissions * 1_000 * 1_000  # kg → g→ mg

        results.append({
            "run": run_id,
            "energy_mWh": energy_mwh,
            "energy_J": energy_j,
            "emissions_mgCO2eq": emissions_g,
            # "duration_s": emissions.duration
        })

finally:
    tracker.stop()

# --- All runs ---
df_all = pd.DataFrame(results)

# --- Valid runs (ignore first & last 2) ---
df_valid = df_all.iloc[DISCARD_FIRST: TOTAL_RUNS - DISCARD_LAST]

# --- Statistics ---
mean = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].mean()
std  = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].std()

summary = pd.DataFrame(
    [mean, std],
    index=["mean", "std"]
).round(2)

# --- Final table ---
df_final = pd.concat([
    df_valid.set_index("run").round(2),
    summary
])

df_final


[codecarbon WARNING @ 02:30:14] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 02:30:14] [setup] RAM Tracking...
[codecarbon INFO @ 02:30:14] [setup] CPU Tracking...
[codecarbon WARNING @ 02:30:15] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist, and are readable, at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 02:30:15] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz
[codecarbon WARNING @ 02:30:15] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 02:30:15] [setup] GPU Tracking...
[codecarbon INFO @ 02:30:15] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:30:15] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
         

,energy_mWh,energy_J,emissions_mgCO2eq
2,0.01,38.43,1.05
3,0.01,37.49,1.02
4,0.01,38.22,1.04
5,0.01,38.34,1.05
6,0.01,43.30,1.18
7,0.01,38.72,1.06
8,0.01,37.85,1.03
9,0.01,37.71,1.03
10,0.01,37.75,1.03
11,0.01,42.71,1.17


In [20]:
a

NameError: name 'a' is not defined

### 3. Defining the Pytorch Lightning Trainer Configuration

We will define the PyTorch Lightning Trainer configuration for fine-tuning the model. Save as for the pre-training.

In [ ]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='val_loss',
    mode='min',
    save_last=True
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='diet-finetune', version=execution_id)

## Trainer
trainer = L.Trainer(
    max_epochs=100,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the fine-tuning pipeline (and running the training)

We will create a `SimpleLightningPipeline` to fine-tune the model. Save as for the pre-training.

In [ ]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


** Seed set to: 42 **
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/logs/run_20250326-183000/run_2025-03-26-18-31-07dbcf7a91.yaml



  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | CNN_PF_Backbone  | 46.6 K | train
1 | fc       | MLP              | 99.2 K | train
2 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
99.2 K    Trainable params
46.6 K    Non-trainable params
145 K     Total params
0.583     Total estimated model params size (MB)
23        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Using DataLoader with shuffle=False


Using DataLoader with shuffle=True                                         


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 99: 100%|██████████| 1/1 [00:01<00:00,  0.82it/s, v_num=3000, val_loss=0.502, val_acc=1.000, train_loss=0.285, train_acc=0.828] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:01<00:00,  0.74it/s, v_num=3000, val_loss=0.502, val_acc=1.000, train_loss=0.285, train_acc=0.828]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/logs/run_20250326-183000/run_2025-03-26-18-31-07dbcf7a91.yaml


## Evaluating the Fine-tuned Model

After fine-tuning the TS2Vec encoder on the DAGHAR dataset, we will evaluate the model's performance on the test set. We create a simple evaluation pipeline, that will:
1. Run forward on test set.
2. Calculcate the metrics. This is specified in the `classification_metrics` dictionary, where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.
3. Perform model analysis, such as plot t-sne embeddings.

The test pipeline requires the following arguments:
- `model`: The fine-tuned model to evaluate.
- `trainer`: The PyTorch Lightning Trainer configuration.
- `log_dir`: The directory to save the evaluation logs.
- `seed`: The random seed for reproducibility.
- `classification_metrics`: A dictionary where the keys are the names of the metrics and the values are the functions that calculate the metrics. The metrics function should use `torchmetrics` API to calculate the metrics.
- `model_analysis`: A function that performs model analysis, such as plotting t-sne embeddings.

Finally, we run the evaluation pipeline with the test data module to evaluate the model on the test set. We set the `task` parameter of the `run` method to `evaluate` to evaluate the model on the test set.

In [ ]:
test_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42,
    classification_metrics={
        "accuracy": torchmetrics.Accuracy(num_classes=6, task="multiclass"),
        "f1": torchmetrics.F1Score(num_classes=6, task="multiclass"),
        "precision": torchmetrics.Precision(num_classes=6, task="multiclass"),
        "recall": torchmetrics.Recall(num_classes=6, task="multiclass"),
        "balanced_accuracy": BalancedAccuracy(num_classes=6, task="multiclass"),
    },
    apply_metrics_per_sample=False,
    # model_analysis={
    #     "tsne": TSNEAnalysis(
    #         height=800,
    #         width=800,
    #         legend_title="Activity",
    #         title="t-SNE of CPC Finetuned on MotionSense",
    #         output_filename="tsne_cpc_finetuned_motionsense.pdf",
    #         label_names={
    #             0: "sit",
    #             1: "stand",
    #             2: "walk",
    #             3: "stair up",
    #             4: "stair down",
    #             5: "run",
    #             6: "stair up and down",
    #         },
    #     )
    # },
)

test_pipeline.run(
    data_module, task="evaluate", ckpt_path=checkpoint_callback.best_model_path
)

** Seed set to: 42 **


Restoring states from the checkpoint path at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/checkpoints/epoch=99-step=100-v1.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/checkpoints/epoch=99-step=100-v1.ckpt


Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/logs/run_20250326-183000/run_2025-03-26-18-33-08a2d7098a.yaml
Using DataLoader with shuffle=False
Using DataLoader with shuffle=False


Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 144.46it/s]
Running classification metrics...
Metrics saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/logs/run_20250326-183000/metrics_2025-03-26-18-33-08a2d7098a.yaml
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/diet/logs/run_20250326-183000/run_2025-03-26-18-33-08a2d7098a.yaml


/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/analysis/metrics/balanced_accuracy.py:59: UserWarning: y_pred contains nan values and not all classes passed
  warnings.warn(f"y_pred contains nan values and not all classes passed")


{'classification': {'accuracy': [0.7430555820465088],
  'f1': [0.7430555820465088],
  'precision': [0.7430555820465088],
  'recall': [0.7430555820465088],
  'balanced_accuracy': [0.8075096011161804]}}